In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import pandas as pd
import numpy as np

2024-07-08 15:38:30.141763: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
## load data
x_train = pd.read_csv("../../../data/05/dii/x_train_diet.csv", index_col=0, header=0)
y_train = pd.read_csv("../../../data/05/dii/y_train.csv", index_col = 0, header = 0 )
sw_train = pd.read_csv("../../../data/05/dii/sw_train.csv", index_col = 0, header = 0 )
x_test = pd.read_csv("../../../data/05/dii/x_test_diet.csv", index_col=0, header=0)
y_test = pd.read_csv("../../../data/05/dii/y_test.csv", index_col = 0, header = 0 )
x_test = x_test[x_train.columns] # reordered columns to match (column ordered changed during taxahfe)

x_train = x_train.sort_index(axis = 0)
y_train = y_train.sort_index(axis = 0)
sw_train = sw_train.sort_index(axis = 0)
x_test = x_test.sort_index(axis = 0)
y_test = y_test.sort_index(axis = 0)

x_train = x_train.reset_index()
y_train = y_train.reset_index()
sw_train = sw_train.reset_index()
x_test = x_test.reset_index()
y_test = y_test.reset_index()

x_train = x_train.drop(columns='SEQN')
y_train = y_train.drop(columns='SEQN')
sw_train = sw_train.drop(columns='SEQN')
x_test = x_test.drop(columns='SEQN')
y_test = y_test.drop(columns='SEQN')

x_train = np.array(x_train)
y_train = np.array(y_train)
sample_weights = np.array(sw_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [3]:
keras.utils.set_random_seed(0)

In [4]:
acc = keras.metrics.Accuracy()

In [5]:
# baseline model
def model_builder(hp):
	# create model
    model = keras.Sequential()
    model.add(keras.layers.BatchNormalization(input_shape=(1,)))

	# Tune the number of units in the first Dense layer
    hp_units_1 = hp.Int('units_1', min_value=32, max_value=256, step=16)
    model.add(keras.layers.Dense(units=hp_units_1, activation='relu'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

	# Compile model
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), sample_weight_mode=sample_weights, metrics=['accuracy'])
    return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='tune',
                     project_name='binary_class_dii_diet')

Reloading Tuner from tune/binary_class_dii_diet/tuner0.json


In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
lr_curve = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.1,
    patience=5)

In [8]:
tuner.search(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early, lr_curve])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_1')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 128 and the optimal learning rate for the optimizer
is 0.01.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2)

acc_per_epoch = history.history['val_accuracy']
best_epoch = acc_per_epoch.index(max(acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
259/259 [==============================] - 1s 2ms/step - loss: 0.7209 - accuracy: 0.5033 - val_loss: 0.6993 - val_accuracy: 0.4860
Epoch 2/50
259/259 [==============================] - 0s 2ms/step - loss: 0.7047 - accuracy: 0.4980 - val_loss: 0.7141 - val_accuracy: 0.5097
Epoch 3/50
259/259 [==============================] - 0s 2ms/step - loss: 0.7031 - accuracy: 0.4964 - val_loss: 0.6962 - val_accuracy: 0.5087
Epoch 4/50
259/259 [==============================] - 0s 1ms/step - loss: 0.6990 - accuracy: 0.5065 - val_loss: 0.7061 - val_accuracy: 0.4956
Epoch 5/50
259/259 [==============================] - 0s 1ms/step - loss: 0.6992 - accuracy: 0.5033 - val_loss: 0.6939 - val_accuracy: 0.4981
Epoch 6/50
259/259 [==============================] - 0s 1ms/step - loss: 0.6953 - accuracy: 0.4975 - val_loss: 0.6912 - val_accuracy: 0.5397
Epoch 7/50
259/259 [==============================] - 0s 1ms/step - loss: 0.6946 - accuracy: 0.5062 - val_loss: 0.6953 - val_accuracy: 0.4777
Epoch 

In [10]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(x_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/40
259/259 [==============================] - 1s 2ms/step - loss: 0.7276 - accuracy: 0.5015 - val_loss: 0.7044 - val_accuracy: 0.4647
Epoch 2/40
259/259 [==============================] - 0s 2ms/step - loss: 0.7028 - accuracy: 0.4993 - val_loss: 0.7047 - val_accuracy: 0.5169
Epoch 3/40
259/259 [==============================] - 0s 1ms/step - loss: 0.7010 - accuracy: 0.4972 - val_loss: 0.6935 - val_accuracy: 0.4889
Epoch 4/40
259/259 [==============================] - 0s 2ms/step - loss: 0.6981 - accuracy: 0.5037 - val_loss: 0.7022 - val_accuracy: 0.4777
Epoch 5/40
259/259 [==============================] - 0s 2ms/step - loss: 0.6971 - accuracy: 0.5045 - val_loss: 0.7023 - val_accuracy: 0.4724
Epoch 6/40
259/259 [==============================] - 0s 2ms/step - loss: 0.6947 - accuracy: 0.5005 - val_loss: 0.6889 - val_accuracy: 0.5198
Epoch 7/40
259/259 [==============================] - 0s 2ms/step - loss: 0.6939 - accuracy: 0.5034 - val_loss: 0.6948 - val_accuracy: 0.4811
Epoch 

In [11]:
b_acc = keras.metrics.BinaryAccuracy(name="binary_accuracy", dtype=None, threshold=0.5)

In [12]:
pred = hypermodel(x_test)
b_acc.update_state(y_test, pred)
result = b_acc.result()
result.numpy()

0.50658405

In [13]:
# save model
hypermodel.save('model/binary_class_dii_diet.keras')